# Bill Cobb's Vehicle Dynamics Professionals' July 2024 Challenge - #3

Given the results of a constant radius understeer test and various speed increments, compute the understeer gradient and cornering compliance evolution with lateral acceleration. The vehicle is a generic car with non-linear steering and non-linear tires simulated with Bill Cobb's BZ3 simulation program.


In [ ]:
%matplotlib widget

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import scipy as sp

In [ ]:
plt.style.use("seaborn-v0_8")

## Part 1: Define constants

We are given known quantities of the vehicle under test. Define them as constants.


In [ ]:
VEHICLE_WB = 2.745  # m
VEHICLE_SR = 20.0  # rad/rad
VEHICLE_WF = 1000  # kg
VEHICLE_WR = 600  # kg
VEHICLE_M = VEHICLE_WF + VEHICLE_WR
VEHICLE_LA = VEHICLE_WB * (VEHICLE_WR / VEHICLE_M)
VEHICLE_LB = VEHICLE_WB * (VEHICLE_WF / VEHICLE_M)

Use constants for column header names to make it easier to access.


In [ ]:
# Provided columns
COLUMN_TIME = "TIME, sec"
COLUMN_LATACC = "LATACC, g"
COLUMN_RUN = "RUN, RUN"
COLUMN_BETA = "SIDSLP, deg"
COLUMN_SPEED = "SPEED, kph"
COLUMN_STEER = "STEER, deg"
COLUMN_YAWVEL = "YAWVEL, deg/sec"

# Computed time domain columns
COLUMN_LATACC_SI = "LATACC, m/s^2"
COLUMN_BETA_SI = "BETA, rad"
COLUMN_SPEED_SI = "SPEED, m/s"
COLUMN_STEER_SI = "STEER, rad"
COLUMN_DELTA_SI = "DELTA, rad"
COLUMN_YAWVEL_SI = "YAWVEL, rad/sec"

# Math channels
COLUMN_DELTA = "DELTA, rad"
COLUMN_BETA_GAIN = "BETAGAIN, rad/rad"
COLUMN_YAWVEL_GAIN = "YAWVELGAIN, rad/s/rad"
COLUMN_DF = "DF, rad/G"
COLUMN_DR = "DR, rad/G"
COLUMN_USG = "USG, rad/G"

Define common conversion factors.


In [ ]:
STD_G = 9.81
DEG2RAD = np.pi / 180.0
RAD2DEG = 1 / DEG2RAD
KPH2MPS = 1000 / 3600.0
MPS2KPH = 1 / RAD2DEG

## Part 1: Data Wrangling


### Load in the data


Read in the data and sanitize it for consumption. The data has the following form:

| Row number | Description                                                                       | Sample                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| ---------- | --------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| Line 1     | Comment header describing the contents of the CSV and relevant vehicle parameters | `"BZ3 Nonlinear Vehicle Dynamics Simulation SR= 20.00 WB=2745 mm  SR=20.00  WF=1000  WR=600"`                                                                                                                                                                                                                                                                                                                                           |
| Line 2     | Column header names and units                                                     | `"TIME, sec";"LATACC, g";"RUN, RUN";"SIDSLP, deg";"SPEED, kph";"STEER, deg";"YAWVEL, deg/sec";                                                                                                                                                                                                                                                                                                                                       ;` |
| Line 3+    | Time series data                                                                  | `1.080    ;0.412    ;14.000   ;-0.498   ;85.000   ;32.611   ;9.702     `                                                                                                                                                                                                                                                                                                                                                                |


In [ ]:
FILE_PATH = "../../data/marc3.txt"

In [ ]:
df = pl.read_csv(FILE_PATH, skip_rows=1, columns=range(7), separator=";").with_columns(
    pl.col("*").str.extract(r"(^-?\d+[.\d]*)").str.to_decimal()
)
df

### Convert to SI units

Perform all calculations in SI units.


In [ ]:
df = df.with_columns(
    (pl.col(COLUMN_LATACC).cast(pl.Float32) * STD_G).alias(COLUMN_LATACC_SI),
    (pl.col(COLUMN_BETA).cast(pl.Float32) * DEG2RAD).alias(COLUMN_BETA_SI),
    (pl.col(COLUMN_SPEED).cast(pl.Float32) * KPH2MPS).alias(COLUMN_SPEED_SI),
    (pl.col(COLUMN_STEER).cast(pl.Float32) * DEG2RAD).alias(COLUMN_STEER_SI),
    (pl.col(COLUMN_YAWVEL).cast(pl.Float32) * DEG2RAD).alias(COLUMN_YAWVEL_SI),
    (pl.col(COLUMN_STEER).cast(pl.Float32) * DEG2RAD / VEHICLE_SR).alias(COLUMN_DELTA),
)
df

### Visualize provided data

Plot the timeseries to see what we are working with.


In [ ]:
fig, axes = plt.subplots(5, 1, sharex=True)

# Plot
columns = [COLUMN_SPEED, COLUMN_STEER, COLUMN_BETA, COLUMN_LATACC, COLUMN_YAWVEL]
for ax, col in zip(axes, columns):
    for name, data in df.group_by(COLUMN_RUN):
        ax.plot(data[COLUMN_TIME], data[col], label=f"Run {name}")
    ax.set_ylabel(col)

# Labels
axes[-1].set_xlabel(COLUMN_TIME)

# Title
fig.suptitle(
    "Bill Cobb's Vehicle Dynamics Professionals' Challenge #3\nOn centre chirp steer frequency response test data"
)

plt.tight_layout()
plt.show()

## Part 2: Data exploration

Given that this is a constant radius test, what is the radius of the skidpad?


### Extract the steady-state response from each run and compute gain


In [ ]:
# Steady-state dataframe (sdf)
sdf = df.group_by(COLUMN_RUN).tail(1).sort(by=COLUMN_RUN)

sdf = sdf.with_columns(
    (pl.col(COLUMN_BETA_SI) / pl.col(COLUMN_DELTA)).alias(COLUMN_BETA_GAIN),
    (pl.col(COLUMN_YAWVEL_SI) / pl.col(COLUMN_DELTA)).alias(COLUMN_YAWVEL_GAIN),
)
sdf

## Part 3: Understeer Gradient and Cornering Compliance

**Approach 1**: The approach employed by Cobb is to compute the gradients
$\frac{d\delta}{da_y}$ and $\frac{d\beta}{da_y}$ which represent the understeer
gradient and rear cornering compliance.

**Approach 2**: Exploit knowledge of the steady-state gain of the sideslip by
steer and yaw rate by steer transfer function to derive $D_f$ and $D_r$.

WARNING. This does not work as it assumes the vehicle is linear across it's
operating region.

Recall that the steady-state gain for the sideslip by steer response is a follows.

$$\frac{\beta(0)}{\delta(0)} = \frac{a_0}{b_0} = \quad \frac{g l_{a} l_{b} m \left(- D_{r} u^{2} + g l_{b}\right)}{D_{f} D_{r} I u^{2} \left(l_{a} + l_{b}\right)} \cdot \frac{ D_{f} D_{r} I u^{2} \left(l_{a} + l_{b}\right)}{\left(g l_{a} l_{b} m\right) \left(D_{f} u^{2} - D_{r} u^{2} + g l_{a} + g l_{b}\right)}$$
$$\frac{\beta(0)}{\delta(0)} = \frac{- D_{r} u^{2} + g l_{b}}{D_{f} u^{2} - D_{r} u^{2} + g l_{a} + g l_{b}}$$

For the yaw rate by steer response.

$$\frac{R(0)}{\delta(0)} = \quad \frac{g^2 l_{a} l_{b} m }{D_{f} D_{r} I u \left(l_{a} + l_{b}\right)} \cdot \frac{D_{f} D_{r} I u^{2} \left(l_{a} + l_{b}\right)}{\left(g l_{a} l_{b} m\right) \left(D_{f} u^{2} - D_{r} u^{2} + g l_{a} + g l_{b}\right)}$$
$$\frac{R(0)}{\delta(0)} = \frac{g u}{D_{f} u^{2} - D_{r} u^{2} + g l_{a} + g l_{b}}$$

Given two equations and two unknowns.

$$AX = B$$

$$
\begin{bmatrix}
\frac{\beta(0)}{\delta(0)} u^2 & (1 - \frac{\beta(0)}{\delta(0)}) u^2 \\
\frac{R(0)}{\delta(0)} u^2 & -\frac{R(0)}{\delta(0)} u^2
\end{bmatrix}
\begin{bmatrix}
D_f \\
D_r
\end{bmatrix}
=
\begin{bmatrix}
g \left(l_b - \frac{\beta(0)}{\delta(0)}\left(l_a + l_b \right)\right) \\
g \left(u - \frac{R(0)}{\delta(0)} \left( l_a + l_b \right)  \right)
\end{bmatrix}
$$

Solve for unknowns $[D_f, D_r]^T$.

$$X = A^{-1}B$$

$$
\begin{bmatrix}
D_f \\
D_r
\end{bmatrix}
=
\begin{bmatrix}
\frac{1}{u^2} & \frac{1 - \frac{\beta(0)}{\delta(0)}}{\frac{R(0)}{\delta(0)} u^2} \\
\frac{1}{u^2} & - \frac{\frac{\beta(0)}{\delta(0)}}{\frac{R(0)}{\delta(0)} u^2}
\end{bmatrix}
\begin{bmatrix}
g \left(l_b - \frac{\beta(0)}{\delta(0)}\left(l_a + l_b \right)\right) \\
g \left(u - \frac{R(0)}{\delta(0)} \left( l_a + l_b \right)  \right)
\end{bmatrix}
$$

$$
\begin{bmatrix}
D_f \\
D_r
\end{bmatrix}
=
\begin{bmatrix}
\frac{g(-\frac{\beta(0)}{\delta(0)}u - \frac{R(0)}{\delta(0)}l_a + u)}{\frac{R(0)}{\delta(0)} u^2} \\
\frac{g(-\frac{\beta(0)}{\delta(0)}u + \frac{R(0)}{\delta(0)l_b})}{\frac{R(0)}{\delta(0)} u^2}
\end{bmatrix}
$$

The understeer gradient is the difference between front and rear cornering compliance.

$$ K = D_f - D_r $$


### Calculate $D_f$, $D_r$, and $K$


In [ ]:
sdf = sdf.with_columns(
    (
        (
            STD_G
            * (
                -pl.col(COLUMN_BETA_GAIN) * pl.col(COLUMN_SPEED_SI)
                - pl.col(COLUMN_YAWVEL_GAIN) * VEHICLE_LA
                + pl.col(COLUMN_SPEED_SI)
            )
        )
        / (pl.col(COLUMN_YAWVEL_GAIN) * pl.col(COLUMN_SPEED_SI) ** 2)
    ).alias(COLUMN_DF),
    (
        (
            STD_G
            * (
                -pl.col(COLUMN_BETA_GAIN) * pl.col(COLUMN_SPEED_SI)
                + pl.col(COLUMN_YAWVEL_GAIN) * VEHICLE_LB
            )
        )
        / (pl.col(COLUMN_YAWVEL_GAIN) * pl.col(COLUMN_SPEED_SI) ** 2)
    ).alias(COLUMN_DR),
)

In [ ]:
sdf = sdf.with_columns((pl.col(COLUMN_DF) - pl.col(COLUMN_DR)).alias(COLUMN_USG))
sdf

## Part 4: Visualize the results


In [ ]:
fig, ax = plt.subplots()

ax.plot(
    sdf[COLUMN_LATACC], sdf[COLUMN_USG, COLUMN_DF, COLUMN_DR] * RAD2DEG, marker="o", label=[r"USG, $K$ [deg/G]", r"$D_f$ [deg/G]", r"$D_r$ [deg/G]"]
)
ax.set_xlabel("Lateral Acceleration [g]")
ax.set_ylabel("Cornering Compliance & Understeer Gradient [deg/G]")
ax.set_ylim(bottom=-1.5, top=10.5)

plt.suptitle(
    r"$\mathbf{Bill\ Cobb's\ Vehicle\ Dynamics\ Professionals'\ Challenge\ \#3}$"
    + "\n"
    + "Constant radius variable speed understeer gradient test"
)

fig.tight_layout()
plt.show()